In [1]:
from datetime import timedelta
from IPython.display import YouTubeVideo

start = int(timedelta(hours = 1, minutes = 27, seconds = 30).total_seconds())
YouTubeVideo("sPiWg5jSoZI", width = 750, height = 562, start = start)

非常棒的參考資料: 

1. https://docs.python.org/3/reference/datamodel.html (官方文件)
2. http://www.cafepy.com/article/python_types_and_objects/python_types_and_objects.html
  - [archive](http://archive.is/IHmZe)
3. https://github.com/dboyliao/Metaprogramming_With_Examples.git

First Look:

![](img/MetaClass.png)

<center>
<a href=http://www.cafepy.com/article/python_types_and_objects/python_types_and_objects.html#types_map>圖片來源</a>
</center>

- `class` 是 `type` 的實例。
- `__new__` 與 `__init__` 必須要有一樣的 api
- 所有 metaclass 皆為 `type` 的子類別。(簡而言之，所有 metaclass 必須繼承 `type`)
- metaclass 的 `__new__` 與 `__init__` 之 api 大致上沒有變化，因為要遷就 `type`。

----

一個物件的生命週期: (例如要創造一個 `A` 的實例 `a`。)

1. python 會呼叫 A.\_\_new\_\_ 創造一個物件 self 。
2. 這個 self 會被 pass 給 A.\_\_init\_\_ 的第一個參數使用。 (**若 A.\_\_new\_\_ 沒有回傳值，則 A.\_\_init\_\_ 不會被呼叫。**)
3. 執行 \_\_init\_\_ 進行物件初始化，並把此物見參照給變數 `a`
4. del a 則可刪除 a 的物件參照，接者可能執行 garbage collection。


一個 class 的生命週期: (例如創造一個 `BMeta` 的實例 `B`，`BMeta` 為一個 metaclass，`B` 為一個 class)

1. python 呼叫 BMeta.\_\_new\_\_(mcl, name, bases, attrs) 創造一個新的物件(新的 class)，並回傳之。(mcl 為 BMeta, bases 為一個 tuple，表示新生成之 class 的繼承對象，attrs 為一個包含所有 class attributes 與 methods 的 dict)
2. 由 BMeta.\_\_new\_\_ 所回傳的物件會被指定給 BMeta.\_\_init\_\_ 的第一個參數 cls。
3. 執行 BMeta.\_\_init\_\_ 進行 cls 的初始化，並把初始化的結果參照給 `B`。
4. del B 可刪除 B 。

- About `super()`: http://stackoverflow.com/questions/9056955/what-does-super-in-new

In [2]:
class A(object):
    _fields = []
    
    def __new__(cls, *args, **kwargs):
        print("__new__ in A is called") 
        print("cls in A: ", cls) # Note that cls need not to be A.
        print("The argumets:", args, kwargs) # We can access the args and kwargs here which will be later passed to __init__
        obj = super(A, cls).__new__(cls)
        obj.name = "qmal"
        return obj

    def __init__(self, *args, **kwargs):
        for key, val in kwargs.items():
            self.__dict__[key] = val
        
        to_add = [a for a in self._fields]
        for key in kwargs.keys():
            to_add.remove(key)
        
        for key, val in zip(to_add, args):
            self.__dict__[key] = val

In [3]:
A(1, 1)

__new__ in A is called
('cls in A: ', <class '__main__.A'>)
('The argumets:', (1, 1), {})


In [4]:
class B(A):
    _fields = ["x", "y"]
    
    def __init__(self, *args, **kwargs):
        print("__init__ in {0} is called".format(type(self).__name__))
        super(B, self).__init__(*args, **kwargs)

In [5]:
b = B(y = 1, x = 1) # 因為 B 沒有重新實做 __new__ ，所以 A.__new__ 被呼叫。

__new__ in A is called
('cls in A: ', <class '__main__.B'>)
('The argumets:', (), {'y': 1, 'x': 1})
__init__ in B is called


In [6]:
b.name

'qmal'

In [7]:
b.x

1

In [8]:
b.y

1

In [9]:
class C(object):
    def __new__(cls):
        print("C.__new__")
        print(id(super(C, cls).__new__), id(object.__new__))
        obj = super(C, cls).__new__(cls)
        return obj

class D(object):
    def __new__(cls):
        print("D.__new__")
        print(id(super(D, cls).__new__), id(object.__new__))
        obj = super(D, cls).__new__(cls)
        return obj

class E(D):
    def __new__(cls):
        print("E.__new__")
        print(id(super(E, cls).__new__), id(D.__new__))
        obj = super(E, cls).__new__(cls)
        return obj

class F(C, D):
    def __new__(cls):
        print("F.__new__")
        print(id(super(F, cls).__new__), id(C.__new__), id(D.__new__))
        obj = super(F, cls).__new__(cls)
        return obj

In [10]:
C()
D()
E()
F()

C.__new__
(4432937040, 4432937040)
D.__new__
(4432937040, 4432937040)
E.__new__
(4463913032, 4463913032)
D.__new__
(4432937040, 4432937040)
F.__new__
(4463913152, 4463913152, 4463913032)
C.__new__
(4463913032, 4432937040)
D.__new__
(4432937040, 4432937040)


In [11]:
# you can make __new__ do the work for __init__ but it is not a recommended python styple.
class G(C):
    def __new__(cls):
        obj = super(G, cls).__new__(cls)
        obj.a = 1
        obj.b = 2
        return obj

In [12]:
g = G()
g.a

C.__new__
(4432937040, 4432937040)


1

In [13]:
g.b

2

----

Metaclass 的 Use case:

當你必須控制所有 class 的行為，而且這些行為不能因繼承被覆寫掉時。

In [22]:
class MyMeta(type): 
    
    def __new__(mcl, name, bases, attrs):
        print("__new__ in {0} is called.".format(mcl.__name__))
        print("Metaclass: ", mcl)
        print("Class Name: ", name)
        print("Base Classes: ", bases)
        print("Attributs and Methods: \n", attrs)
        print 
        clsobj = super(MyMeta, mcl).__new__(mcl, name, bases, attrs)
        return clsobj
    
    def __init__(cls, name, bases, attrs):
        
        print("__init__ in {0} is called".format(type(cls).__name__))
        print("New Class Created: ", cls)
        print("Metaclass: ", type(cls))
        cls.x = 1

In [23]:
class C(A):
    __metaclass__ = MyMeta
    # A 會被傳給 bases
    # metaclass 會被傳給 mcl
    # 接下來定義的東西會被傳給 attrs
    
    def __new__(cls, y, z):
        print("__new__ in {0} is called.".format(cls.__name__))
        print id(super(C, cls).__new__), id(A.__new__), id(object.__new__)
        obj = super(C, cls).__new__(cls)
        return obj
    
    def __init__(self, y, z):
        print("__init__ in {0} is called.".format(type(self).__name__))
        self.y = y
        self.z = z
        
    def fun(self, x):
        return x + 1

__new__ in MyMeta is called.
('Metaclass: ', <class '__main__.MyMeta'>)
('Class Name: ', 'C')
('Base Classes: ', (<class '__main__.A'>,))
('Attributs and Methods: \n', {'fun': <function fun at 0x10a5f2c80>, '__module__': '__main__', '__metaclass__': <class '__main__.MyMeta'>, '__new__': <function __new__ at 0x10a5f2b90>, '__init__': <function __init__ at 0x10a5f2c08>})

__init__ in MyMeta is called
('New Class Created: ', <class '__main__.C'>)
('Metaclass: ', <class '__main__.MyMeta'>)


In [24]:
c = C(1, 2)

__new__ in C is called.
4463912672 4463912672 4432937040
__new__ in A is called
('cls in A: ', <class '__main__.C'>)
('The argumets:', (), {})
__init__ in C is called.


In [18]:
c.name # A 的 __new__ 加進來的

'qmal'

In [19]:
c.x # MyMeta 那兒搞來的

1

----

以下我們來看兩種生成一個 class D 的方式，

完全是等價的。

In [11]:
D = MyMeta.__new__(MyMeta, "D", (A, ), {'__qualname__': "D", '__module__': '__main__'})
MyMeta.__init__(D, "D", (A, ), {})
d1 = D()

__new__ in MyMeta is called.
Metaclass:  <class '__main__.MyMeta'>
Class Name:  D
Base Classes:  (<class '__main__.A'>,)
Attributs and Methods: 
 {'__qualname__': 'D', '__module__': '__main__'}

__init__ in MyMeta is called
New Class Created:  <class '__main__.D'>
Metaclass:  <class '__main__.MyMeta'>

__new__ in A is called
cls in A:  D


In [12]:
class D(A, metaclass = MyMeta):
    pass
d2 = D()

__new__ in MyMeta is called.
Metaclass:  <class '__main__.MyMeta'>
Class Name:  D
Base Classes:  (<class '__main__.A'>,)
Attributs and Methods: 
 {'__qualname__': 'D', '__module__': '__main__'}

__init__ in MyMeta is called
New Class Created:  <class '__main__.D'>
Metaclass:  <class '__main__.MyMeta'>

__new__ in A is called
cls in A:  D


以上兩種寫法等價。

In [13]:
print(d1.x, d1.name)

1 qmal


In [14]:
print(d2.x, d2.name)

1 qmal
